<a href="https://colab.research.google.com/github/pansilup/is713_grp2/blob/master/is713_group2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IS 713 Group 2

## 1. Introduction

In [ ]:
# add the EDA of the data
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1oQQAU542s8wPHNJNmNBICaMDh75AiXBVPJagODpWz4Q/edit#gid=1250765839')
sheet = wb.worksheet('final_dataset')
data = sheet.get_all_values()

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df = pd.DataFrame(data)
df.columns = df.iloc[0]
df = df.iloc[1:]
# df["year"] = pd.to_numeric(df["year"])
df.replace('', np.nan, inplace=True)
df.dropna(inplace=True)
df.reset_index(drop=True,inplace=True)

df['consumer_price_index'] = df['consumer_price_index'].astype('float32')
df

,no,country,year,fertility_rate,consumer_price_index,female-labourFrc-percentage,dragon_year,tiger_year,chinese_prominance
0,1,Singapore,1990,1.83,71.816734,37.5420903,0,0,1
1,1,Singapore,1991,1.73,74.276962,37.6152636,0,0,1
2,1,Singapore,1992,1.72,75.957901,37.732357,0,0,1
3,1,Singapore,1993,1.74,77.696808,38.0359161,0,0,1
4,1,Singapore,1994,1.71,80.105507,38.1750817,0,0,1
...,...,...,...,...,...,...,...,...,...
265,9,Myanmar,2015,2.198,129.322678,41.6164699,0,0,0
266,9,Myanmar,2016,2.182,138.283218,41.100912,0,0,0
267,9,Myanmar,2017,2.168,144.606262,40.5913159,0,0,0
268,9,Myanmar,2018,2.154,154.544083,41.0267118,0,0,0


In [ ]:
!pip install --upgrade plotly
!pip install jupyter-dash

import plotly
plotly.__version__

In [ ]:
import plotly.express as px
fig = px.line(df, x="year", y="fertility_rate", color="country")
fig.show()

In [ ]:
import plotly.express as px
fig = px.line(df, x="year", y="consumer_price_index", color="country")
fig.show()

In [ ]:
import plotly.express as px
fig = px.line(df, x="year", y="consumer_price_index", color="chinese_prominance", facet_col="country")
                  #hover_data=['female-labourFrc-percentage'])
fig.show()

In [ ]:
import plotly.express as px
fig = px.scatter(df, x="year", y="fertility_rate", color="country", size='consumer_price_index',)
                  #hover_data=['female-labourFrc-percentage'])
fig.show()

In [ ]:
import plotly.express as px
df_sg = df[df['country']=='Singapore']
fig = px.scatter(df_sg, x="chinese_x_dragon", y="fertility_rate", color="country",)
                 #size='female-labourFrc-percentage', hover_data=['female-labourFrc-percentage'])
fig.show()

## 2. Models

In [ ]:
# Use standard errors that are clustered by country
# experiment with time and entity fixed effects

#%%Model_1
#          Fertility_rate_it = A ‘Country-with-chinese-culture_i’ x ‘Dragon-year_t’ +  C ‘consumer-price-index_it’ + err

# #%%Model_2
#           Fertility_rate_it = B ‘Country-with-chinese-culture_i’ x Tiger-year_t’ + C ‘consumer-price-index_it’ + err

# #%%Model_3
#           Fertility_rate_it = A ‘Country-with-chinese-culture_i’ x ‘Dragon-year_t’ + B ‘Country-with-chinese-culture_i’ x Tiger-year_t’ + C ‘consumer-price-index_it’ + err

In [ ]:
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1oQQAU542s8wPHNJNmNBICaMDh75AiXBVPJagODpWz4Q/edit#gid=1250765839')
sheet = wb.worksheet('final_dataset')
data = sheet.get_all_values()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df = pd.DataFrame(data)
df.columns = df.iloc[0]
df = df.iloc[1:]
# df["year"] = pd.to_numeric(df["year"])
df.replace('', np.nan, inplace=True)
df.dropna(inplace=True)
df.reset_index(drop=True,inplace=True)
df 

,no,country,year,fertility_rate,consumer_price_index,female-labourFrc-percentage,dragon_year,tiger_year,chinese_prominance
0,1,Singapore,1990,1.83,71.8167331,37.5420903,0,0,1
1,1,Singapore,1991,1.73,74.2769602,37.6152636,0,0,1
2,1,Singapore,1992,1.72,75.9579006,37.732357,0,0,1
3,1,Singapore,1993,1.74,77.6968046,38.0359161,0,0,1
4,1,Singapore,1994,1.71,80.1055086,38.1750817,0,0,1
...,...,...,...,...,...,...,...,...,...
265,9,Myanmar,2015,2.198,129.322672,41.6164699,0,0,0
266,9,Myanmar,2016,2.182,138.283214,41.100912,0,0,0
267,9,Myanmar,2017,2.168,144.606264,40.5913159,0,0,0
268,9,Myanmar,2018,2.154,154.544082,41.0267118,0,0,0


In [ ]:






#%% PP
## Final models start here ...

In [ ]:
!pip install linearmodels

In [ ]:
import pandas as pd
import statsmodels.formula.api as sm
from linearmodels import PanelOLS

exceldata = pd.ExcelFile('sample_data/final3.xlsx')
df = exceldata.parse('final_dataset')
#print(df)

df['chinese_x_dragon'] = df['chinese_prominance']*df['dragon_year']
df['chinese_x_tiger'] = df['chinese_prominance']*df['tiger_year']
df3 = df.set_index(['country', 'year'])

## Model 1


In [ ]:
model_1_0 = PanelOLS.from_formula('fertility_rate ~ consumer_price_index+EntityEffects',data=df3).fit(cov_type='clustered', cluster_entity=True, cluster_time=True)
print(model_1_0)

In [ ]:
model_1_1 = PanelOLS.from_formula('fertility_rate ~ dragon_year+tiger_year+consumer_price_index+EntityEffects',data=df3).fit(cov_type='clustered', cluster_entity=True, cluster_time=True)
print(model_1_1)

In [ ]:
model_1_2 = PanelOLS.from_formula('fertility_rate ~ consumer_price_index+dragon_year+tiger_year+chinese_x_tiger+chinese_x_dragon',data=df3).fit(cov_type='clustered', cluster_entity=True, cluster_time=True)
print(model_1_2)

## Model 2


In [ ]:
model_2_0 = PanelOLS.from_formula('change_in_fertility_rate_subtrt ~ change_in_cons_prc_idx_subtrt+EntityEffects',data=df3).fit(cov_type='clustered', cluster_entity=True, cluster_time=True)
print(model_2_0)

In [ ]:
model_2_1 = PanelOLS.from_formula('change_in_fertility_rate_subtrt ~ change_in_cons_prc_idx_subtrt+dragon_year+tiger_year+EntityEffects',data=df3).fit(cov_type='clustered', cluster_entity=True, cluster_time=True)
print(model_2_1)

In [ ]:
model_2_2 = PanelOLS.from_formula('change_in_fertility_rate_subtrt ~ change_in_cons_prc_idx_subtrt+dragon_year+tiger_year+chinese_x_dragon+chinese_x_tiger+EntityEffects',data=df3,drop_absorbed=True).fit(cov_type='clustered', cluster_entity=True, cluster_time=True)
print(model_2_2)

## Model 3


In [ ]:
model_3_0 = PanelOLS.from_formula('change_in_fertility_rate_mvg_avg ~ change_in_cons_prc_idx_mvg_avg+EntityEffects',data=df3).fit(cov_type='clustered', cluster_entity=True, cluster_time=True)
print(model_3_0)

In [ ]:
model_3_1 = PanelOLS.from_formula('change_in_fertility_rate_mvg_avg ~ change_in_cons_prc_idx_mvg_avg+dragon_year+tiger_year+EntityEffects',data=df3).fit(cov_type='clustered', cluster_entity=True, cluster_time=True)
print(model_3_1)

In [ ]:
model_3_2 = PanelOLS.from_formula('change_in_fertility_rate_mvg_avg ~ change_in_cons_prc_idx_mvg_avg+dragon_year+tiger_year+chinese_x_dragon+chinese_x_tiger+EntityEffects',data=df3,drop_absorbed=True).fit(cov_type='clustered', cluster_entity=True, cluster_time=True)
print(model_3_2)